In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
from glob import glob
import json

xwikis = sorted(glob('../summarization/filtered-en*.splitted.translated'))
xwikis

['../summarization/filtered-en00.splitted.translated',
 '../summarization/filtered-en01.splitted.translated',
 '../summarization/filtered-en02.splitted.translated',
 '../summarization/filtered-en03.splitted.translated',
 '../summarization/filtered-en04.splitted.translated',
 '../summarization/filtered-en05.splitted.translated']

In [3]:
import malaya
import json
import re
from glob import glob
from tqdm import tqdm
from malaya.text.rouge import postprocess_summary, filter_rouge, _get_word_ngrams, _rouge_clean, cal_rouge
from malaya.text.function import split_into_sentences
import random

In [4]:
def remove_parenthesis(string, substring, lower_case=True):
    removed = []
    for r in re.finditer(r'\([^()]*\)', string):
        span = r.span()
        subs = string[span[0]: span[1]]
        if substring in (subs.lower() if lower_case else subs):
            removed.append(span)

    if len(removed):
        selected = [string[:removed[0][0]]]
        for i in range(0, len(removed) - 1, 2):
            selected.append(string[removed[i][1]: removed[i + 1][0]])

        selected.append(string[removed[-1][1]:])
        return re.sub(r'[ ]+', ' ', ' '.join(selected)).strip()
    else:
        return string
    
def cut_article(article, maxlen = 900):
    splitted = malaya.text.function.split_into_sentences(article, minimum_length = 2)
    temp = []
    for s in splitted:
        temp.append(s)
        if len(' '.join(temp).split()) > maxlen:
            break
    
    return ' '.join(temp)

def filtering_rouge(summary, contents, start = 0.15, increment = 0.05, break_at = 900):
    filtered = contents[:]
    i = 0
    while len(filtered.split()) > break_at:
        filtered = filter_rouge(summary, filtered, n = 1, threshold = start + increment * i)
        i += 1
    return filtered

In [5]:
before, after = [], []
for f in xwikis:
    limit = random.randint(20000, 30000)
    count = 0
    with open(f) as fopen:
        for l in tqdm(fopen):
            data = json.loads(l)

            contents = ' '.join(data['translated-article'])
            summary = remove_parenthesis(data['translated-summary'], 'jadual di bawah')
            
            if len(set(contents.split())) <= 20 or len(set(summary.split())) <= 20:
                continue
            
            t = malaya.text.function.split_into_sentences(summary)
            
            if len(t) > 2:
            
                before.append(t)
                after.append(contents)
                
                count += 1
                
            if count >= limit:
                break

19723it [00:03, 5118.20it/s]
23478it [00:05, 4307.66it/s]
36449it [00:08, 4454.48it/s]
29775it [00:08, 3472.90it/s]
35222it [00:05, 6275.29it/s]
28575it [00:04, 6785.13it/s]


In [6]:
before[-1], after[-1]

(['Maghribi adalah anggota PBB dan tergolong dalam Kesatuan Afrika, Liga Arab, Arab Maghreb Union (UMA), Organisasi Kerjasama Islam (OIC), Gerakan Bukan Berasingan dan Komuniti Negara Sahel-Sahara (CEN_SAD).',
  'Hubungan Maghribi sangat berbeza antara negara Afrika, Arab, dan Barat.',
  'Maghribi mempunyai hubungan yang kuat dengan Barat untuk mendapatkan faedah ekonomi dan politik.',
  'Perancis dan Sepanyol tetap menjadi rakan dagang utama, serta pemiutang utama dan pelabur asing di Maghribi.',
  'Dari jumlah pelaburan asing di Maghribi, Kesatuan Eropah melabur sekitar 73.5%, sedangkan, dunia Arab hanya melabur 19.3%.',
  'Banyak negara dari wilayah Teluk dan Maghreb Parsi semakin terlibat dalam projek pembangunan berskala besar di Maghribi.'],
 'Maghribi mempunyai sejarah pemerintahan raja. Sebagai contoh, raja Maghribi pada tahun 1965 menggantung parlimen dan memerintah sebagai diktator selama dua tahun. Hubungan luar negeri dengan kuasa barat menjadi tegang akibat ini. Memeluk Ma

In [7]:
with open('article.json', 'w') as fopen:
    json.dump({'before': before, 'after': after}, fopen)